In [40]:
import pandas as pd 
import matplotlib.pyplot as plt 
import time 
import numpy as np 
import seaborn as sns 
import datetime
import warnings
warnings.filterwarnings('ignore')

#### Data Cleaning and Preprocessing
#### Stores with wrong Locations Coordinates are removed.    

In [3]:
#INITIAL DATASET
initial_dataset = pd.read_csv('H:\intern_lovelocal\ML_Project\store_product_orders_data_modified.csv',float_precision = "%.9f")
#PREPROCESSING(REMOVING THE FALSE DATA FROM THE DATAFRAME)
initial_dataset.drop( initial_dataset[ initial_dataset['store_latitude'] == 0 ].index , inplace=True)
initial_dataset.drop( initial_dataset[ initial_dataset['store_longitude'] == 0 ].index , inplace=True)
initial_dataset.drop( initial_dataset[ initial_dataset['store_latitude'] < 0 ].index , inplace=True)
initial_dataset.drop( initial_dataset[ initial_dataset['store_longitude'] < 0 ].index , inplace=True)
#DATASET CONTAINING UNIQUE STORE IDS ONLY
unique_store_id_df = pd.read_excel('H:\intern_lovelocal\ML_Project\lat_long_google_api.xlsx')
unique_store_id_df = unique_store_id_df[['store_id']]

In [6]:
initial_dataset.head()

,product_id,product_name,order_id,order_product_version,orders_version,customer_id,barcode,store_id,store_name,store_category,product_category_name,order_date,delivery_address,delivery_status,product_mrp,store_latitude,store_longitude,customer_latitude,customer_longitude
0,35655,Gemini Refined Sunflower Oil Jar (5 L),64890091,0,0,4941c8cacc20474cae212b33518e1fe5,8906006330054,189559,b'Sarvodaya Supermarket',Grocers,Edible Oils & Ghee,2019-12-01 02:58:11.242573,"{""flatNumber"": ""05"", ""googleAddress"": ""Vanita ...",DELIVERED,700.00,19.020993,72.840382,19.025638,72.836223
1,33199,Aashirvaad Superior MP Atta (1 kg),64890091,0,0,4941c8cacc20474cae212b33518e1fe5,8901725121747,189559,b'Sarvodaya Supermarket',Grocers,"Foodgrains, Oil & Masala",2019-12-01 02:58:11.242573,"{""flatNumber"": ""05"", ""googleAddress"": ""Vanita ...",DELIVERED,52.00,19.020993,72.840382,19.025638,72.836223
2,55775,Mix Dryfruits 500 Gm,31384697,0,0,4b8e778049aa4801b33978eba68e863d,NaN,138106,"b'Kanha Sweets,Snacks & Dry Fruit (Borivali)'",Sweets & Farsan Shops,Rice & Rice Products,2019-12-01 07:06:46.397832,"{""flatNumber"": ""2206"", ""googleAddress"": ""Shop ...",DELIVERED,310.00,19.235378,72.864190,19.235231,72.864125
3,36768,McCain Smiles (1.25 kg),89309898,1,1,4e5cbd6b7660492286cf5934a651384d,8906000610893,189559,b'Sarvodaya Supermarket',Grocers,Dairy,2019-12-01 06:33:10.835154,"{""flatNumber"": ""117/29"", ""googleAddress"": ""356...",DELIVERED,230.85,19.020993,72.840382,19.020916,72.840370
4,106533,Svt Amla Fizz 300Ml,89309898,1,1,4e5cbd6b7660492286cf5934a651384d,8904223300140,189559,b'Sarvodaya Supermarket',Grocers,Beverages,2019-12-01 06:33:10.835154,"{""flatNumber"": ""117/29"", ""googleAddress"": ""356...",DELIVERED,20.00,19.020993,72.840382,19.020916,72.840370


In [5]:
initial_dataset.dtypes

product_id                 int64
product_name              object
order_id                   int64
order_product_version      int64
orders_version             int64
customer_id               object
barcode                   object
store_id                   int64
store_name                object
store_category            object
product_category_name     object
order_date                object
delivery_address          object
delivery_status           object
product_mrp              float64
store_latitude           float64
store_longitude          float64
customer_latitude        float64
customer_longitude       float64
dtype: object

In [24]:
#CONVETING DATA IN DATE FROM STRING TO DATETIME OBJECT
initial_dataset['order_date']= pd.to_datetime(initial_dataset['order_date'])

# Explaining the parameters of the raw dataset from which the desired features are extracted

#### order_id or Transaction Id:  associated with each transaction, Order_id of each transaction is unique. 
#### customer_id: Id assocoated to each of the customers. Each customer Id is Unique and it represents  the Custmer.
#### store_id: Unique for each store, represents a particluar store. 
#### store_category: Division of store on the basis of type of goods sold(Grocery, Cosmetics, Sweets etc).
#### product_category_name: The Category of the product. 
#### order_date: The date when the product was ordered.
#### delivery_address: Text Address of the address of the customer. 
#### delivery_status: Order Delivered/ Order Cancelled. 
#### store_latitude, store_longitude: coordinates of store location.
#### customer_latitude, customer_longitude: Coordinates of customer Location.



# Feature_calculation


## All The features are calculated at a store Level

### 1. Total orders 
### 2. Average orders per day 
### 3. Average gap of time between subsequent orders
### 4. Ratio between delivery to pickup orders 
### 5. Total Pickup Orders
### 6. Total orders delivered
### 7. Average delivery distance 
### 8. Average basket size 
### 9. Average basket Price
### 10. Average Unique Basket size 
### 11. Average Unique Basket Price
### 12. Average Diffrenece Between range of basket price per day 
### 13. Average number of Customers
### 14. Top 5 products with Maximum sale
### 15. Bottom 5 products with minimum sale
### 16. Average orders per day per store
### 17. Type of Goods Category Sold

### Total orders per store

In [25]:
      #FEATURE CALCULATION
      #CALCULATING TOTAL ORDERS PER STORE
      def total_orders(store_id):
          #MAKING DEEP COPY OF ORIGINAL DATASET
          initial_dataset_copy = initial_dataset.copy()
          #PIVOTING STORE_ID
          initial_dataset_copy = initial_dataset_copy.pivot_table(index=['store_id'], aggfunc='size').sort_values(ascending= False)
          #Converting from series to dataframe
          initial_dataset_copy = initial_dataset_copy.to_frame()
          #NAming third column
          initial_dataset_copy = initial_dataset_copy.rename(columns = {0: "total_number_of_orders_per_store"})
          #Setting the Index
          initial_dataset_copy.reset_index(inplace= True)
          initial_dataset_copy.head()
          rslt_df = initial_dataset_copy[(initial_dataset_copy['store_id'] == store_id)]
          #Setting the Index
          rslt_df.reset_index(inplace= True)
          freq_total_orders = rslt_df['total_number_of_orders_per_store'][0]
           
          if rslt_df.empty:
             return  0
          else:
             return  freq_total_orders

### Average orders per day per store

In [26]:
      #FEATURE CALCULATION
      #CALCULATING AVERAGE ORDERS PER STORE
      def average_orders(store_id):
          #MAKING DEEP COPY OF ORIGINAL DATASET 
          initial_dataset_copy = initial_dataset.copy()
          #CALCULATION OF NUMBER OF DAYS THE STORE HAS PASSED BEING ON BOARD WITH LOVELOCAL
          rslt_df = initial_dataset_copy.loc[(initial_dataset_copy['store_id'] == store_id)]
          min_date= min(rslt_df['order_date'])
          max_date= max(rslt_df['order_date'])
          difference = max_date- min_date
          difference= difference.days
          #CALCULATION OF AVERAGE NUMBER OF ORDERS PER STORE
          if(difference!=0):
             average_orders_per_store =float( total_orders(store_id)/(difference))
             return average_orders_per_store
          else:
             return 0


 ### Average gap of time between subsequent orders

In [27]:
      #FEATURE CALCULATION
      #CALCULATING TIME LAPSE  B/W ORDERS 
      def average_gap_of_time_bwtn_subsequent_orders(store_id):
          #MAKING DEEP COPY OF ORIGINAL DATASET 
          initial_dataset_copy = initial_dataset.copy()
          rslt_df = initial_dataset_copy.loc[(initial_dataset_copy['store_id'] == store_id)]
          rslt_df = rslt_df.sort_values(by='order_date',ascending=False)
          rslt_df = rslt_df[['order_date']]
          #Setting the Index
          rslt_df = rslt_df.reset_index(drop=True)
          #SUBTRACTING CONSECUTIVE ROWS 
          rslt_df["time_gap"] = (rslt_df["order_date"].diff(-1))
          
          for i in range(len(rslt_df["time_gap"])-1):
               #rslt_df["time_gap"][i] = rslt_df["order_date"][i].days*24 + rslt_df["order_date"][i].hours
               rslt_df["time_gap"][i] = (rslt_df["time_gap"][i].total_seconds())/3600.0
               
          Total = rslt_df['time_gap'].mean()
          return Total

### Ratio between delivery to pickup orders, Total Orders delivered, total orders pickup

In [28]:
      #FEATURE CALCULATION
      #CALCULATING OF RATIO BETWEEN DELIVERED AND PICKUP
      def ratio_delivery_pickup(store_id):
          initial_dataset_copy = initial_dataset.copy()
          rslt_df = initial_dataset_copy.loc[(initial_dataset_copy['store_id'] == store_id)]
          delivered_count= rslt_df["delivery_address"].count()
          pick_up =rslt_df["delivery_address"].isnull().sum()
          if(pick_up!=0):
             ratio_delivered_pick_up = delivered_count/pick_up
             return ratio_delivered_pick_up,delivered_count,pick_up
          else:
             return 0,delivered_count,pick_up

### Average delivery distance

In [29]:
      #FEATURE CALCULATION
      #CALCULATING OF AVERAGE DISTANCE BETWEEN STORE AND THE CUSTOMERS RESIDENCE 
      def avg_delivery_distance(store_id):
          initial_dataset_copy = initial_dataset.copy()
          rslt_df = initial_dataset_copy.loc[(initial_dataset_copy['store_id'] == store_id)]
          #DROPPING THE NULL ROWS
          rslt_df = rslt_df[rslt_df['customer_longitude'].notna()]
          #DELETING 0 CUSTOMER LONGITUDE AND LATITUDE
          rslt_df = rslt_df[rslt_df.customer_longitude != 0]
          rslt_df = rslt_df[rslt_df.customer_latitude != 0]
          #SELECTING REQUIRED COLUMNS
          rslt_df = rslt_df[['customer_latitude','customer_longitude','store_latitude','store_longitude','delivery_status']]
          #CREATING A COLUMN
          rslt_df['distance'] = pd.Series(index=rslt_df.index)
   
          #Setting the Index
          rslt_df = rslt_df.reset_index(drop=True)
          #CALCULATION OF DISTANCE 
          from math import sin, cos, sqrt, atan2, radians
          for i in range(len(rslt_df['customer_longitude'])):
      
                # approximate radius of earth in km
                R = 6373.0
                
                lat1 = radians(rslt_df['customer_latitude'][i])
                lon1 = radians(rslt_df['customer_longitude'][i])
                lat2 = radians(rslt_df['store_latitude'][i])
                lon2 = radians(rslt_df['store_longitude'][i])
                dlon = lon2 - lon1
                dlat = lat2 - lat1
                a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
                c = 2 * atan2(sqrt(a), sqrt(1 - a))
                distance = R * c
                rslt_df['distance'][i] =distance*1000.0 
                k = distance*1000.0 
       
          
          #rslt_df_less_200 = rslt_df[rslt_df['distance'] < 6000] 
          #rslt_df_less_200 = rslt_df[rslt_df['distance']]
          Total = rslt_df['distance'].mean()
          #Total = rslt_df_less_200['distance'].mean()
          return Total

### Average Basket size and Prive

In [30]:
      #FEATURE CALCULATION
      #CALCULATION OF AVERAGE BASKET SIZE AND PRICE
      
      def avg_basket_size_and_price(store_id):
          initial_dataset_copy = initial_dataset.copy()
          rslt_df = initial_dataset_copy.loc[(initial_dataset_copy['store_id'] == store_id)] 
          #GROUPING BY ORDER_ID
          groups = rslt_df.groupby('order_id')
          #TOTAL BASKETS    
          basket_size= groups['product_id'].count()
          #AVERAGE BASKET SIZE
          avg_basket_size = basket_size.mean()
          #TOTAL PRICE OF BASKET   
          basket_price= groups['product_mrp'].apply(lambda x: sum(x))    
          #AVERAGE PRICE OF BASKET
          avg_basket_price = basket_price.mean()
          return avg_basket_size, avg_basket_price
       

### Average Unique Basket size and Prive

In [31]:
      #FEATURE CALCULATION
      #CALCULATION OF AVERAGE UNIQUE BASKET SIZE AND PRICE
      def avg_unique_basket_size_and_price(store_id):    
          initial_dataset_copy = initial_dataset.copy()
          rslt_df = initial_dataset_copy.loc[(initial_dataset_copy['store_id'] == store_id)] 
          rslt_df = rslt_df.drop_duplicates(['order_id', 'product_id'])
          #GROUPING BY ORDER_ID
          groups = rslt_df.groupby('order_id')
          #TOTAL BASKETS    
          basket_size= groups['product_id'].count()
          #AVERAGE BASKET SIZE
          avg_unique_basket_size = basket_size.mean()
          #TOTAL PRICE OF BASKET   
          basket_price= groups['product_mrp'].apply(lambda x: sum(x))    
          #AVERAGE PRICE OF BASKET
          avg_unique_basket_price = basket_price.mean()
          return avg_unique_basket_size, avg_unique_basket_price

### Average Difference between range of basket price

In [32]:
      #FEATURE CALCULATION
      #CALCULATING Average DIFFERENCE BETWEEN RANGE OF PRICE OF BASKET ON DAILY BASIS
      def diff_btwn_range_of_basket_price(store_id):
          initial_dataset_copy = initial_dataset.copy()
          rslt_df = initial_dataset_copy.loc[(initial_dataset_copy['store_id'] == store_id)] 
          #GROUPING BY ORDER_ID
          groups = rslt_df.groupby('order_id')
          #TOTAL PRICE OF BASKET   
          basket_price= groups['product_mrp'].apply(lambda x: sum(x))
          basket_price = basket_price.to_frame()
          basket_price.reset_index(inplace= True)
          basket_price = basket_price.rename(columns = {"product_mrp": "basket_price"})
          #LEFT JOIN TO ADD DATES
          rslt_df_no_dups = rslt_df.drop_duplicates('order_id')
          basket_price = pd.merge(left=basket_price, right=rslt_df_no_dups, left_on='order_id', right_on='order_id')
          basket_price = basket_price[['basket_price','order_date']]
          basket_price['order_date'] = basket_price['order_date'].dt.date
          basket_price = basket_price.sort_values(by='order_date')
          basket_price.reset_index(inplace = True, drop = True) 
          
          #GROUPING BY DATE
          gr = basket_price.groupby('order_date')['basket_price']
          basket_price_modified = (gr.max()-gr.min()).reset_index()
          range_of_basket_price = basket_price_modified['basket_price'].mean()
          return range_of_basket_price

### Average number of customers per day

In [33]:
      #FEATURE CALCULATION
      #AVERAGE CUSTOMERS PER DAY
      def avg_customers(store_id):
          initial_dataset_copy = initial_dataset.copy()
          rslt_df = initial_dataset_copy.loc[(initial_dataset_copy['store_id'] == store_id)] 
          rslt_df['order_date'] = rslt_df['order_date'].dt.date
          #PREVENTING SAME CUSTOMER ORDERS ON SAME DATE WITH THE SAME STORE
          rslt_df_no_dups = rslt_df.drop_duplicates(subset=['order_date', 'customer_id','store_id'], keep='first')
          min_date= min(rslt_df_no_dups['order_date'])
          max_date= max(rslt_df_no_dups['order_date'])
          difference = max_date- min_date
          difference= difference.days
          if (difference!=0):
              avg_no_of_customers = float((rslt_df_no_dups['customer_id'].count())/difference)
              return avg_no_of_customers
          elif (difference==0):
              return 0

### Top 5 products with max sale  

In [34]:
      #FEATURE CALCULATION
      #TOP 5 PRODUCTS WITH MAX SALE PER STORE
      def top_five_products_name_max_sale(store_id):
          initial_dataset_copy = initial_dataset.copy()
          rslt_df = initial_dataset_copy.loc[(initial_dataset_copy['store_id'] == store_id)] 
          rslt_df = rslt_df.pivot_table(index=["product_name"], aggfunc='size').sort_values(ascending= False).to_frame().rename(columns = {0: "freq",})
          #Setting the Index
          rslt_df.reset_index(inplace= True)
          if(len(rslt_df)>=5):
             list_top_5_products = [rslt_df['product_name'][0],rslt_df['product_name'][1],rslt_df['product_name'][2],rslt_df['product_name'][3],rslt_df['product_name'][4]]
          elif(len(rslt_df)==4):
             list_top_5_products = [rslt_df['product_name'][0],rslt_df['product_name'][1],rslt_df['product_name'][2],rslt_df['product_name'][3]]
          elif(len(rslt_df)==3):
             list_top_5_products = [rslt_df['product_name'][0],rslt_df['product_name'][1],rslt_df['product_name'][2]]  
          elif(len(rslt_df)==2):
             list_top_5_products = [rslt_df['product_name'][0],rslt_df['product_name'][1]]
          elif(len(rslt_df)==1):
             list_top_5_products = [rslt_df['product_name'][0]]             

          return list_top_5_products

### Bottom 5 products with min sale  

In [35]:
      #FEATURE CALCULATION
      #LOWEST 5 PRODUCTS WITH LEAST SALE
      def least_five_products_name_max_sale(store_id):
          initial_dataset_copy = initial_dataset.copy()
          rslt_df = initial_dataset_copy.loc[(initial_dataset_copy['store_id'] == store_id)] 
          rslt_df = rslt_df.pivot_table(index=["product_name"], aggfunc='size').sort_values(ascending= False).to_frame().rename(columns = {0: "freq",})
          #Setting the Index
          rslt_df.reset_index(inplace= True)
          if(len(rslt_df)>=5):          
             list_bottom_5_products = [rslt_df['product_name'].iloc[-1],rslt_df['product_name'].iloc[-2],rslt_df['product_name'].iloc[-3],rslt_df['product_name'].iloc[-4],rslt_df['product_name'].iloc[-5]]
          elif(len(rslt_df)==4):          
             list_bottom_5_products = [rslt_df['product_name'].iloc[-1],rslt_df['product_name'].iloc[-2],rslt_df['product_name'].iloc[-3],rslt_df['product_name'].iloc[-4]]
          elif(len(rslt_df)==3):          
             list_bottom_5_products = [rslt_df['product_name'].iloc[-1],rslt_df['product_name'].iloc[-2],rslt_df['product_name'].iloc[-3]]
          elif(len(rslt_df)==2):          
             list_bottom_5_products = [rslt_df['product_name'].iloc[-1],rslt_df['product_name'].iloc[-2]]
          elif(len(rslt_df)==1):          
             list_bottom_5_products = [rslt_df['product_name'].iloc[-1]]   

          return list_bottom_5_products

### Category of Goods sold by store

In [36]:
      #FEATURE CALCULATION
      #CATEGORY OF GOOD SOLD MOST BY THE STOREs
      def freq_goods_category_sold(store_id):
          initial_dataset_copy = initial_dataset.copy()
          rslt_df = initial_dataset_copy.loc[(initial_dataset_copy['store_id'] == store_id)]
          rslt_df = rslt_df.pivot_table(index=["product_category_name"], aggfunc='size').sort_values(ascending= False).to_frame().rename(columns = {0: "freq",})
          rslt_df.reset_index(inplace= True)
          freq_goods_category_sold= rslt_df['product_category_name'][0]
          return freq_goods_category_sold

## Main function to calculate all the features at a store level

In [41]:
feature_vectors ={} 
def feature_vectors_calculation(store_id):
      feature_vectors ={} 

         
      #CALLING FUNCTION 
      #INSERTING VALUE IN THE DICTIONARY
      feature_vectors["total_orders"] = total_orders(store_id)
      #################################################################################################

      #CALLING FUNCTION 
      #INSERTING VALUE IN THE DICTIONARY
      feature_vectors["average_orders_per_day"] = average_orders(store_id)
      ###################################################################################################

      
      #CALLING FUNCTION
      #INSERTING VALUE IN THE DICTIONARY
      feature_vectors["average_gap_of_time_in_hours_in_subsequent_orders"] = average_gap_of_time_bwtn_subsequent_orders(store_id)
      ##########################################################################################

      
      
      #CALLING FUNCTION 
      #INSERTING VALUE IN THE DICTIONARY
      ratio_delivered_to_pick_up, total_orders_delivered, total_orders_pickup= ratio_delivery_pickup(store_id)
      feature_vectors["ratio_delivered_to_pick_up"] = ratio_delivered_to_pick_up
      feature_vectors["total_orders_delivered"] = total_orders_delivered
      feature_vectors["total_orders_pickup"] = total_orders_pickup
      ############################################################################################

      
      #CALLING FUNCTION
      #INSERTING VALUE IN THE DICTIONARY
      feature_vectors["avg_delivery_distance_in_metre"] = avg_delivery_distance(store_id)
      #################################################################################

      #CALLING FUNCTION
      #INSERTING VALUE IN THE DICTIONARY
      avg_basket_size, avg_basket_price= avg_basket_size_and_price(store_id)
      feature_vectors["avg_basket_size"] = avg_basket_size
      feature_vectors["avg_basket_price"] = avg_basket_price
      
      ################################################################################

      
      #CALLING FUNCTION
      #INSERTING VALUE IN THE DICTIONARY
      avg_unique_basket_size, avg_unique_basket_price = avg_unique_basket_size_and_price(store_id)    
      feature_vectors["avg_unique_basket_size"] = avg_unique_basket_size
      feature_vectors["avg_unique_basket_price"] = avg_unique_basket_price
      ############################################################################s####
        

    
      #######################################################


      #CALLING FUNCTION
      #INSERTING VALUE IN THE DICTIONARY
      feature_vectors["average_diff_btwn_range_of_basket_price"] = diff_btwn_range_of_basket_price(store_id)
      ##########################################################

      
      #CALLING FUNCTION
      #INSERTING VALUE IN THE DICTIONARY
      feature_vectors["avg_no_of_customers_per_day"] = avg_customers(store_id)
      #############################################################

      
      #CALLING FUNCTION
      #INSERTING VALUE IN THE DICTIONARY
      feature_vectors["top_five_products_name_max_sale"] = top_five_products_name_max_sale(store_id)
      ##########################################################

      #CALLING FUNCTION
      #INSERTING VALUE IN THE DICTIONARY
      feature_vectors["list_bottom_5_products"] = least_five_products_name_max_sale(store_id)
      #############################################################

      
      #CALLING FUNCTION
      #INSERTING VALUE IN THE DICTIONARY
      feature_vectors["freq_goods_category_sold"] = freq_goods_category_sold(store_id)
      ##########################################################
      return feature_vectors



In [42]:
#INITIALISING THE DATAFRAME BY NAMING THE COLUMNS ACCORDING TO SPECIFIC FEATURES
df_cat_1 = pd.DataFrame(columns = ['total_orders','average_orders_per_day','average_gap_of_time_in_hours_in_subsequent_orders','ratio_delivered_to_pick_up','total_orders_delivered','total_orders_pickup','avg_delivery_distance_in_metre','avg_basket_size','avg_basket_price','avg_unique_basket_size','avg_unique_basket_price','average_diff_btwn_range_of_basket_price','avg_no_of_customers_per_day','top_five_products_name_max_sale','list_bottom_5_products','freq_goods_category_sold']) 

# Loop Calculating all the features for all the stores stored in a dataframe

In [43]:
for store_id in unique_store_id_df['store_id']:
    raw_data_types= feature_vectors_calculation(store_id)
    df_temp  = pd.DataFrame([raw_data_types], columns=raw_data_types.keys())
    df_cat_1= df_cat_1.append(df_temp)


In [45]:
#RESETTING THE INDEX
df_cat_1.reset_index(drop=True, inplace= True)
#ADDING COLUMN FOR THE STORE IDS
df_cat_1['store_id'] = unique_store_id_df['store_id'].values 
df_cat_1.head()

,total_orders,average_orders_per_day,average_gap_of_time_in_hours_in_subsequent_orders,ratio_delivered_to_pick_up,total_orders_delivered,total_orders_pickup,avg_delivery_distance_in_metre,avg_basket_size,avg_basket_price,avg_unique_basket_size,avg_unique_basket_price,average_diff_btwn_range_of_basket_price,avg_no_of_customers_per_day,top_five_products_name_max_sale,list_bottom_5_products,freq_goods_category_sold,store_id
0,6712,58.877193,0.408229,190.771429,6677,35,122.089756,3.116063,332.816063,3.073816,330.660074,935.849462,18.657895,"[Amul Butter (500 g), Toor Dal (1 kg), Amul Bu...",[ Glucon D Energy Drink Pure Glucose Tangy Ora...,"Foodgrains, Oil & Masala",189559
1,38,0.487179,50.711917,0.000000,38,0,315.390632,2.111111,185.333333,2.111111,185.333333,147.571429,0.230769,"[Dairy Milk (13.8 g), Kinder joy , Mota's Pota...","[Amul Shrikhand Badam Pista (500 Gm), Special ...",All products,138106
2,934,5.247191,4.581109,92.400000,924,10,2352.956707,1.074799,376.498274,1.074799,376.498274,346.289474,4.831461,"[Chocolate Truffle Small Cake, German Chocolat...","[Mousse Chocolate, Paneer Pav Bhaji Roll, Germ...",Cakes & Pastries,68629
3,2353,13.219101,1.819067,15.116438,2207,146,521.754089,2.758499,260.164549,2.737397,258.917186,399.451797,4.623596,"[Gold Winner Sunflower Oil (1 L), Sugar (1 kg)...","[Aachi Appalam 100Grm, Krishna Ragi Dosai Flou...",All products,974401
4,306,2.684211,9.035357,0.000000,306,0,1072.804944,6.120000,564.400000,6.120000,564.400000,182.051282,0.434783,"[Mother Dairy Milk Pouch (500 ml), Tata Salt (...",[Lifebuoy Cool Fresh Hand Wash Refill (185 ml)...,Rice & Rice Products,67421


#### Getting total number of Unique stores

In [49]:
df_cat_1.shape #Getting total number of stores 

(1020, 17)

#### Saving dataframe as an Excel file in the system

In [ ]:
   
#EXPORTING AS AN EXCEL FILE
df_cat_1.to_excel(r'H:\intern_lovelocal\ML_project/response_category_1_data_new_new.xlsx', index = False)